In [8]:
!pip install -U pip
%pip install -Uq "unstructured[all-docs]" Pillow lxml
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai
%pip install -Uq python_dotenv
!apt-get update
!apt-get install -y poppler-utils

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/p

In [13]:
#Need for Colab
#!pip install -q --upgrade "Pillow<11"

In [2]:
#For Local Run
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
#For Colab Run
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
from unstructured.partition.pdf import partition_pdf
import os

path = os.path.join(os.getcwd(), "drive/MyDrive/Colab Notebooks/RAG/RRMenu.pdf")

chunks = partition_pdf(
    filename=path,                                                              # only required parameter
    infer_table_structure=False,                                                 # extract tables
    strategy="auto",                                                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    skip_infer_table_types=True,
    # extract_image_block_types=["Image"],                                      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",                                   # if none, images and tables will be saved in base64

    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",                                               # or "basic" | Menu has clear titles
    max_characters=10000,                                                       # max size of chunk
    combine_text_under_n_chars=2000,                                            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                                                      # start new part after 6000 characters
    )

In [14]:
print(len(chunks))

9


In [25]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '43c2099fd4e91cc48e55834faccbbcbd',
 'text': 'GOURMET FIRE-GRILLED BURGERS\n\nALL BURGERS INCLUDE A BOTTOMLESS SIDE\n\nSteak Fries • Yukon Chips • Steamed Broccoli • Side Salad Sweet Potato Fries (1.79) • Garlic Fries (1.79)\n\nOUR SIGNATURES Burgers you won’t find anywhere else. 100% Fresh, Never Frozen Beef.\n\nTHE SOUTHERN CHARM BURGER® Brown sugar glaze, candied bacon, Whiskey River® BBQ Sauce, Cheddar, caramelized onions, lettuce and mayo on a toasted brioche bun. 14.99 cal 1190\n\nBLACK & BLEU\n\nSautéed and blackened portobello mushrooms, caramelized onions, creamy cheese sauce, Bleu cheese crumbles, lettuce and roasted garlic aioli on a toasted brioche bun. 15.39 cal 980\n\nWHISKEY RIVER® BBQ\n\nDIP IT. DUNK IT. DOUSE YOUR BURGER.\n\nWhiskey River® BBQ Sauce, crispy onion straws, Cheddar, lettuce, tomatoes and mayo on a toasted sesame bun. 13.79 cal 1140 THE MADLOVE BURGER A Cheddar-and-Parmesan crisp, Provolone, Swiss, jalapeño relis

In [17]:
texts = []

for chunk in chunks:
    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [33]:
from langchain_core.documents import Document


# Chunks --> Langchain Documents
documents = []
for chunk in texts:
  documents.append(Document(page_content = chunk.text,
                            metadata=chunk.metadata.to_dict()                   # Document() expects metadata as a dict, not unstrucuted metadata
                            )
  )

print(documents)

[Document(metadata={'file_directory': '/content/drive/MyDrive/Colab Notebooks/RAG', 'filename': 'RRMenu.pdf', 'languages': ['eng'], 'last_modified': '2025-11-26T20:39:29', 'page_number': 1, 'orig_elements': 'eJzlWm1v2zgS/iuEgTvcAaFLUqRE9puduG2uiZO1k110u0VBiSNbF1kyJLnZdHH//YaSnObF3UvukH7wfYn9yDMSqRk+8wyZj38MIIcVFM3nzA1ekwELIUyUUNQyYagMZES1iBl1zEiexDoOQjk4IIMVNNbZxqLPH4OkLCuXFbaBusW5vSk3zeclZItlg1e4EHLI0Kv/4TpzzRKv67C7vC6zovGuHz9KyYb8gOBf+emAbGEoOqgZH6pHsDVGPKhv6gZWfhrn2e+Qz9c2gcG/8Ic0y+GzyypImrK68QavkrJocNqvXJV9gVenN0ft52GZ25hMywbisryqX81Gbwe9f2FX4D1ns1MoNsO1SwftjIrFxi7aiX8cQLEYfGqv1s3nVemyNIP2tQqGr5RzKsILwV4H5rUw3nuNnp+LzSqGyr8mP9QGfvevbPD27HJ2Orkgb45nE/p2dnxyMjki48vZ28ls7l2bm3U7nHdgHXqj68NQJpCqGCCg2oQJlUmI36IopbGKRMwFgAP+oqEMjI+dDoeqDWUHOTMdVlEwNI9xa75vwRydnGxjR46nhyeXRxMyIuOzi4uz05PJfE7mx0eTu1G9yJocdgVVGh2zKFb48BDXpwaO61PF1CnruEsjrRP2kkFVRg3Ft6D2kDPVYc3CXXgvgzpvwF6RN1UGNfltg/6CfNhclQU5XGbr20veagWOjKsySco8u72eOSBzm1tH5tcADTkvG9uU/f3+xoeR+fvW9q2t8iy599PdZLksEgztoqyyr+Au/Oh2JI6JtQ45posF

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.vectorstores import Chroma
import shutil, os

filtered_docs = filter_complex_metadata(documents)                              # langchain doesnt like unstructureds metadata structure

# create embeddings for vector storage
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")               # huggingface is free

# remove chroma_db so you dont re-store data
shutil.rmtree("./chroma_db", ignore_errors=True)
os.makedirs("./chroma_db", exist_ok=True)

# Create vectorstore
vectorstore = Chroma.from_documents(filtered_docs,
                                    embeddings,
                                    persist_directory="./chroma_db"
                                    )

In [ ]:
# get a tuple of the top k chunks and their scores based on their similarity score (i.e. cosine sim)
results = vectorstore.similarity_search_with_relevance_scores("Can i make a custom pizza?", k=3)

# print top k docs and their score
for doc, score in results:
    print(f"Score:  {score} \nText: {doc.page_content} \n {'-'*100}")

In [ ]:
import transformers, torch, os
from huggingface_hub import login

login(token=os.getenv("HF_TOKEN"))

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},                               # half the size of weight
    device_map="auto"                                                           # tries gpu, then cpu
)


In [30]:
query = "Which burgers are vegetarian?"                                         # user input
context = "\n\n".join(doc.page_content for doc, _ in results[:3])

messages = [
    {"role": "system", "content": "Answer only from the menu below. Be brief."},
    {"role": "user", "content": f"Menu:\n{context}\n\n{query}"}
]

outputs = pipe(
    messages,
    max_new_tokens=150,                                                         # number of tokens to generate
    temperature=0.6,                                                            # more likely to select most probable word (logit / 0.7)
    do_sample=True,                                                             # need to set this to true for temperature to have any effect
    top_p=0.9,                                                                  # model choses from top tokens that make up 90% of the probability mass (ignores super low prob tokens from being selected)
    pad_token_id=pipe.tokenizer.eos_token_id
)

# last element is the llm reply
answer = outputs[0]["generated_text"][-1]["content"]
print(answer)

Impossible™ burger, Veggie burger, and The WORKS burger (with mushrooms and no meat) are vegetarian options.
